In [39]:
from google.colab import files
uploaded = files.upload()

Saving LoanApprovalPrediction.csv to LoanApprovalPrediction (2).csv


In [40]:
import pandas as pd
import numpy as np


In [41]:
data=pd.read_csv('LoanApprovalPrediction.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            598 non-null    object 
 1   Gender             598 non-null    object 
 2   Married            598 non-null    object 
 3   Dependents         586 non-null    float64
 4   Education          598 non-null    object 
 5   Self_Employed      598 non-null    object 
 6   ApplicantIncome    598 non-null    int64  
 7   CoapplicantIncome  598 non-null    float64
 8   LoanAmount         577 non-null    float64
 9   Loan_Amount_Term   584 non-null    float64
 10  Credit_History     549 non-null    float64
 11  Property_Area      598 non-null    object 
 12  Loan_Status        598 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 60.9+ KB


In [43]:
data.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,586.000000,598.000000,598.000000,577.000000,584.000000,549.000000
mean,0.755973,5292.252508,1631.499866,144.968804,341.917808,0.843352
std,1.007751,5807.265364,2953.315785,82.704182,65.205994,0.363800
min,0.000000,150.000000,0.000000,9.000000,12.000000,0.000000
25%,0.000000,2877.500000,0.000000,100.000000,360.000000,1.000000
50%,0.000000,3806.000000,1211.500000,127.000000,360.000000,1.000000
75%,1.750000,5746.000000,2324.000000,167.000000,360.000000,1.000000
max,3.000000,81000.000000,41667.000000,650.000000,480.000000,1.000000


In [44]:
data.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           12
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           21
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

In [45]:
data.shape

(598, 13)

In [46]:
data['Credit_History'].unique()

array([ 1.,  0., nan])

In [47]:
import plotly.express as px
px.bar(data,x='Loan_Status',title='Approve Not_approve chart')

In [48]:
px.box(data,x='LoanAmount')

In [49]:
percentile25=data['LoanAmount'].quantile(0.25)
percentile75=data['LoanAmount'].quantile(0.75)

In [53]:
iqr = percentile75 - percentile25
iqr

67.0

In [54]:
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

In [56]:
new_df_cap = data.copy()

new_df_cap['LoanAmount'] = np.where(
    new_df_cap['LoanAmount'] > upper_limit,
    upper_limit,
    np.where(
        new_df_cap['LoanAmount'] < lower_limit,
        lower_limit,
        new_df_cap['LoanAmount']
    )
)

In [65]:
px.box(new_df_cap,x='LoanAmount')

In [66]:
px.box(new_df_cap,x='CoapplicantIncome')

In [67]:
percentile25=data['CoapplicantIncome'].quantile(0.25)
percentile75=data['CoapplicantIncome'].quantile(0.75)


In [68]:
iqr=percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

In [69]:
data = new_df_cap.copy()

data['CoapplicantIncome'] = np.where(
    data['CoapplicantIncome'] > upper_limit,
    upper_limit,
    np.where(
        data['CoapplicantIncome'] < lower_limit,
        lower_limit,
        data['CoapplicantIncome']
    )
)

In [70]:
px.box(data,x='CoapplicantIncome')

In [71]:
data['Loan_Status'].value_counts()

Y    411
N    187
Name: Loan_Status, dtype: int64

In [72]:
from sklearn.utils import resample
df_majority = data[(data['Loan_Status']=='Y')]
df_minority = data[(data['Loan_Status']=='N')]
df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples= 411,
                                 random_state=42)
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [73]:
df_upsampled.shape

(822, 13)

In [74]:
df_upsampled.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           16
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           37
Loan_Amount_Term     36
Credit_History       56
Property_Area         0
Loan_Status           0
dtype: int64

In [75]:
px.bar(df_upsampled,x='Loan_Status')

In [76]:
df_upsampled.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
332,LP002115,Male,Yes,3.0,Not Graduate,No,2647,1587.0,173.0,360.0,1.0,Rural,N
571,LP002912,Male,Yes,1.0,Graduate,No,4283,3000.0,172.0,84.0,1.0,Rural,N
292,LP001955,Female,No,0.0,Graduate,No,5000,2541.0,151.0,480.0,1.0,Rural,N
35,LP001109,Male,Yes,0.0,Graduate,No,1828,1330.0,100.0,NaN,0.0,Urban,N
345,LP002142,Female,Yes,0.0,Graduate,Yes,5500,0.0,105.0,360.0,0.0,Rural,N


In [77]:
df_upsampled.drop(columns=['Loan_ID'],axis=1,inplace=True)

In [79]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
df_upsampled['Gender']=l.fit_transform(df_upsampled['Gender'])
df_upsampled['Education']=l.fit_transform(df_upsampled['Education'])
df_upsampled['Married']=l.fit_transform(df_upsampled['Married'])
df_upsampled['Self_Employed']=l.fit_transform(df_upsampled['Self_Employed'])
df_upsampled['Property_Area']=l.fit_transform(df_upsampled['Property_Area'])
df_upsampled['Loan_Status']=l.fit_transform(df_upsampled['Loan_Status'])


In [83]:
df_upsampled['Property_Area'].value_counts(normalize=True)

1    0.356448
2    0.326034
0    0.317518
Name: Property_Area, dtype: float64

In [80]:
df_upsampled.head()
# male=1 female=0
#married_yes=1 no=0
#not_graduate=1 graduate=0
# self_employed=1 not=0
#rural=0 urban=2 semi_urban=1
#loan status no=0 yes=1

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
332,1,1,3.0,1,0,2647,1587.0,173.0,360.0,1.0,0,0
571,1,1,1.0,0,0,4283,3000.0,172.0,84.0,1.0,0,0
292,0,0,0.0,0,0,5000,2541.0,151.0,480.0,1.0,0,0
35,1,1,0.0,0,0,1828,1330.0,100.0,NaN,0.0,2,0
345,0,1,0.0,0,1,5500,0.0,105.0,360.0,0.0,0,0


In [84]:
x=df_upsampled.iloc[:,:-1]
y=df_upsampled.iloc[:,-1]

In [87]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [97]:
from sklearn.impute import SimpleImputer
s=SimpleImputer()
s.fit(x_train)
x_train=s.transform(x_train)

In [102]:
x_test=s.transform(x_test)

In [105]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)

RandomForestClassifier()

In [106]:
from sklearn.metrics import accuracy_score
pred=model.predict(x_test)
accuracy_score(pred,y_test)

0.9454545454545454

In [108]:
from sklearn.model_selection import GridSearchCV
cf=GridSearchCV(RandomForestClassifier(),{
    'n_estimators' : [10,20,30],
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'max_depth' : [10,100,50,20],
},cv=5,return_train_score=False)
cf.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [10, 100, 50, 20],
                         'n_estimators': [10, 20, 30]})

In [109]:
cf.best_score_

0.8965417534119823

In [110]:
cf.best_params_

{'criterion': 'gini', 'max_depth': 50, 'n_estimators': 30}

In [111]:
model2=RandomForestClassifier(criterion= 'gini', max_depth = 50, n_estimators=30)
model2.fit(x_train,y_train)

RandomForestClassifier(max_depth=50, n_estimators=30)

In [112]:
p=model2.predict(x_test)
accuracy_score(p,y_test)

0.9454545454545454

In [113]:
import pickle
filename='LoanApprovalPrediction.sav'
pickle.dump(model2,open(filename,'wb'))